<a href="https://colab.research.google.com/drive/1Pib_WNdXi51uKhguP1upPyDvlB7cWCOS?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huấn luyện một mô hình nhận dạng âm thanh đơn giản

Notebook này minh họa cách huấn luyện một mô hình Nhận diện Âm thanh Đơn giản (Simple Audio Recognition) có kích thước chỉ 20 kB để nhận biết các từ khóa trong lời nói.

Mô hình được tạo ra trong notebook này sẽ được sử dụng trong ví dụ micro_speech dành cho TensorFlow Lite for MicroControllers.

<table class="tfo-notebook-buttons" align="left"> <td> <a target="_blank" href="https://colab.research.google.com/drive/1Pib_WNdXi51uKhguP1upPyDvlB7cWCOS?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Chạy trên Google Colab</a> </td> <td> <a target="_blank" href="https://github.com/phuneil1001/My_Profile/blob/main/train_micro_speech_model_phuneil.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Xem mã nguồn trên GitHub</a> </td> </table>

**Đào tạo nhanh hơn nhiều khi sử dụng tăng tốc GPU.** Trước khi tiếp tục, hãy đảm bảo bạn đang sử dụng thời gian chạy GPU bằng cách vào **Thời gian chạy -> Thay đổi loại thời gian chạy** và đặt **Bộ tăng tốc phần cứng: GPU**. Đào tạo 15.000 lần lặp sẽ mất 1,5 - 2 giờ trên thời gian chạy GPU.

## Cấu hình mặc định

**SỬA ĐỔI** các hằng số sau cho trường hợp sử dụng cụ thể của bạn.

In [ ]:
# Một danh sách các từ (phân tách bằng dấu phẩy) mà bạn muốn huấn luyện.
# Các lựa chọn bao gồm: yes,no,up,down,left,right,on,off,stop,go
# Tất cả các từ khác sẽ được dùng để huấn luyện nhãn "unknown" và dữ liệu âm thanh yên lặng (không có từ nào được nói) sẽ dùng để huấn luyện nhãn "silence".
WANTED_WORDS = "yes,no"

# Số bước huấn luyện và các mức learning rate có thể được chỉ định dưới dạng danh sách phân tách bằng dấu phẩy.
# Ví dụ, TRAINING_STEPS=12000,3000 và LEARNING_RATE=0.001,0.0001
# sẽ chạy tổng cộng 12.000 vòng lặp huấn luyện, với learning rate là 0.001 cho 8.000 bước đầu,
# và 0.0001 cho 3.000 bước cuối cùng.
TRAINING_STEPS = "12000,3000"
LEARNING_RATE = "0.001,0.0001"

# Tính tổng số bước huấn luyện, giá trị này sẽ được dùng để xác định tên file checkpoint.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# In ra cấu hình để xác nhận
print("Training these words: %s" % WANTED_WORDS)
print("Training steps in each stage: %s" % TRAINING_STEPS)
print("Learning rate in each stage: %s" % LEARNING_RATE)
print("Total number of training steps: %s" % TOTAL_STEPS)


**DO NOT MODIFY** các hằng số sau vì chúng bao gồm các đường dẫn tệp được sử dụng trong sổ ghi chép này và dữ liệu được chia sẻ trong quá trình đào tạo và suy luận.

In [ ]:
# Tính tỷ lệ mẫu huấn luyện cho 'silence' và 'unknown'
# để đảm bảo số lượng mẫu cho mỗi nhãn là bằng nhau.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # cộng thêm 2 nhãn: 'silence' và 'unknown'
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Các hằng số dùng chung cho cả quá trình huấn luyện và suy luận
PREPROCESS = 'micro'
WINDOW_STRIDE = 20
MODEL_ARCHITECTURE = 'tiny_conv' # Các lựa chọn khác gồm: single_fc, conv,
                      # low_latency_conv, low_latency_svdf, tiny_embedding_conv

# Các hằng số chỉ dùng cho huấn luyện
VERBOSITY = 'WARN'
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Đường dẫn thư mục và file dùng trong huấn luyện
DATASET_DIR =  'dataset/'
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # lưu các checkpoint và file huấn luyện khác.

# Đường dẫn thư mục và file dùng trong suy luận
import os
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
  os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'model.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'model.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'float_model.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'model.cc')
SAVED_MODEL = os.path.join(MODELS_DIR, 'saved_model')

QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN


## Thiết lập Môi trường

Cài đặt Phụ thuộc

In [ ]:
import tensorflow as tf

**XÓA** mọi dữ liệu cũ từ các lần chạy trước

In [ ]:
!rm -rf {DATASET_DIR} {LOGS_DIR} {TRAIN_DIR} {MODELS_DIR}

Clone Repo Github của TensorFlow, nơi chứa mã liên quan cần thiết để chạy hướng dẫn này.

In [ ]:
!git clone -q --depth 1 https://github.com/tensorflow/tensorflow

Tải TensorBoard để trực quan hóa độ chính xác và mất mát trong quá trình đào tạo.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOGS_DIR}

## Huấn luyện

Đoạn mã sau sẽ tải về bộ dữ liệu và bắt đầu quá trình huấn luyện.


In [ ]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--data_dir={DATASET_DIR} \
--wanted_words={WANTED_WORDS} \
--silence_percentage={SILENT_PERCENTAGE} \
--unknown_percentage={UNKNOWN_PERCENTAGE} \
--preprocess={PREPROCESS} \
--window_stride={WINDOW_STRIDE} \
--model_architecture={MODEL_ARCHITECTURE} \
--how_many_training_steps={TRAINING_STEPS} \
--learning_rate={LEARNING_RATE} \
--train_dir={TRAIN_DIR} \
--summaries_dir={LOGS_DIR} \
--verbosity={VERBOSITY} \
--eval_step_interval={EVAL_STEP_INTERVAL} \
--save_step_interval={SAVE_STEP_INTERVAL}

# Bỏ qua quá trình huấn luyện

Nếu bạn không muốn mất một hoặc hai tiếng để huấn luyện mô hình từ đầu, bạn có thể tải về các checkpoint đã được huấn luyện sẵn bằng cách bỏ comment các dòng bên dưới (xóa ký tự ‘#’ ở đầu mỗi dòng) và chạy chúng.


In [ ]:
#!curl -O "https://storage.googleapis.com/download.tensorflow.org/models/tflite/speech_micro_train_2020_05_10.tgz"
#!tar xzf speech_micro_train_2020_05_10.tgz

## Tạo một mô hình TensorFlow để suy luận

Kết hợp các kết quả huấn luyện liên quan (đồ thị, trọng số, v.v.) vào một tệp duy nhất để sử dụng cho việc suy luận. Quá trình này được gọi là "đóng băng" mô hình và mô hình thu được gọi là mô hình/đồ thị đã đóng băng, vì sau quá trình này mô hình không thể được huấn luyện tiếp nữa.


In [ ]:
!rm -rf {SAVED_MODEL}
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--wanted_words=$WANTED_WORDS \
--window_stride_ms=$WINDOW_STRIDE \
--preprocess=$PREPROCESS \
--model_architecture=$MODEL_ARCHITECTURE \
--start_checkpoint=$TRAIN_DIR$MODEL_ARCHITECTURE'.ckpt-'{TOTAL_STEPS} \
--save_format=saved_model \
--output_file={SAVED_MODEL}

## Tạo một mô hình TensorFlow Lite

Chuyển đổi đồ thị đã đóng băng thành một mô hình TensorFlow Lite, mô hình này sẽ được lượng tử hóa hoàn toàn để sử dụng với các thiết bị nhúng.

Ô lệnh sau đây cũng sẽ in ra kích thước của mô hình, kích thước này sẽ dưới 20 kilobyte.


In [ ]:
import sys
# Thêm đường dẫn này để có thể import các mô-đun xử lý giọng nói.
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data  # Nhập module để xử lý dữ liệu đầu vào âm thanh.
import models      # Nhập module chứa các kiến trúc mô hình nhận dạng giọng nói.
import numpy as np # Nhập thư viện numpy để xử lý tính toán số học và ma trận.


In [ ]:
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

DATA_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

In [ ]:
# Cấu hình các tham số cho mô hình nhận dạng từ khóa
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))), # Số lượng nhãn/từ khóa cần nhận dạng
    SAMPLE_RATE,       # Tần số lấy mẫu âm thanh (Hz), ví dụ 16000Hz
    CLIP_DURATION_MS,  # Thời lượng mỗi đoạn audio (tính bằng mili giây)
    WINDOW_SIZE_MS,    # Kích thước cửa sổ phân tích tín hiệu (ms)
    WINDOW_STRIDE,     # Bước trượt của cửa sổ phân tích (ms)
    FEATURE_BIN_COUNT, # Số lượng đặc trưng sau khi trích xuất (ví dụ số lượng MFCC)
    PREPROCESS         # Loại tiền xử lý đặc trưng (vd: MFCC, micro...)
)

# Khởi tạo bộ xử lý âm thanh, chịu trách nhiệm:
# - Tải dữ liệu về (nếu cần)
# - Tiền xử lý, chia tỉ lệ dữ liệu (train/validation/test)
# - Tạo nhãn cho từng file âm thanh
audio_processor = input_data.AudioProcessor(
    DATA_URL,                # Đường dẫn tải dữ liệu gốc (có thể là url Google Speech Commands)
    DATASET_DIR,             # Thư mục lưu dữ liệu local
    SILENT_PERCENTAGE,       # Tỉ lệ mẫu 'silence' (im lặng) trong tập huấn luyện
    UNKNOWN_PERCENTAGE,      # Tỉ lệ mẫu 'unknown' (không phải từ khóa) trong tập huấn luyện
    WANTED_WORDS.split(','), # Danh sách các từ khóa mục tiêu cần nhận dạng
    VALIDATION_PERCENTAGE,   # Tỉ lệ dữ liệu validation (%)
    TESTING_PERCENTAGE,      # Tỉ lệ dữ liệu test (%)
    model_settings,          # Tham số cấu hình mô hình (ở trên)
    LOGS_DIR                 # Thư mục lưu lại các log/quá trình tiền xử lý
)


In [ ]:
with tf.compat.v1.Session() as sess:
    # Tạo bộ chuyển đổi để chuyển mô hình TensorFlow đã lưu thành mô hình TensorFlow Lite (dạng số thực - float).
    float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
    # Thực hiện chuyển đổi mô hình sang định dạng .tflite (dạng số thực).
    float_tflite_model = float_converter.convert()
    # Ghi mô hình đã chuyển đổi ra file nhị phân, lấy tên từ biến FLOAT_MODEL_TFLITE.
    float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
    # In ra kích thước của file mô hình dạng float.
    print("Float model is %d bytes" % float_tflite_model_size)

    # Tiếp tục tạo bộ chuyển đổi mô hình, lần này để chuyển sang dạng lượng tử hóa (quantized).
    converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
    # Thiết lập chế độ tối ưu hóa mặc định (bao gồm lượng tử hóa).
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # Thiết lập kiểu dữ liệu đầu vào và đầu ra là int8 (để lượng tử hóa toàn phần).
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    # Định nghĩa hàm sinh dữ liệu đại diện (representative dataset generator)
    # Hàm này dùng một tập mẫu nhỏ từ dữ liệu thật để hướng dẫn quá trình lượng tử hóa, giúp mô hình giữ được độ chính xác.
    def representative_dataset_gen():
        for i in range(100):
            data, _ = audio_processor.get_data(
                1,                # Số lượng mẫu mỗi lần lấy
                i*1,              # Vị trí bắt đầu lấy mẫu
                model_settings,   # Các tham số cấu hình mô hình
                BACKGROUND_FREQUENCY,
                BACKGROUND_VOLUME_RANGE,
                TIME_SHIFT_MS,
                'testing',        # Lấy từ tập kiểm thử
                sess              # Phiên TensorFlow hiện tại
            )
            # Làm phẳng dữ liệu và chuyển thành float32, định dạng đầu vào phù hợp.
            flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
            yield [flattened_data]
    # Gán generator này cho converter để phục vụ lượng tử hóa.
    converter.representative_dataset = representative_dataset_gen

    # Thực hiện chuyển đổi mô hình đã lượng tử hóa thành định dạng .tflite.
    tflite_model = converter.convert()
    # Ghi mô hình đã lượng tử hóa ra file, tên lấy từ biến MODEL_TFLITE.
    tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
    # In ra kích thước file mô hình đã lượng tử hóa (quantized).
    print("Quantized model is %d bytes" % tflite_model_size)


## Kiểm tra độ chính xác của mô hình TensorFlow Lite

Xác minh rằng mô hình mà chúng ta đã xuất ra vẫn còn chính xác, sử dụng API Python của TensorFlow Lite và bộ dữ liệu kiểm tra của chúng ta.


In [ ]:
# Hàm hỗ trợ để chạy kiểm thử (inference) trên mô hình TFLite
def run_tflite_inference(tflite_model_path, model_type="Float"):
    # Tải dữ liệu kiểm thử (test)
    np.random.seed(0)  # Đặt seed ngẫu nhiên để đảm bảo kết quả kiểm thử có thể lặp lại (reproducible).
    with tf.compat.v1.Session() as sess:
        test_data, test_labels = audio_processor.get_data(
            -1,                  # Lấy toàn bộ mẫu trong tập kiểm thử
            0,                   # Vị trí bắt đầu lấy mẫu
            model_settings,      # Tham số cấu hình mô hình
            BACKGROUND_FREQUENCY,
            BACKGROUND_VOLUME_RANGE,
            TIME_SHIFT_MS,
            'testing',           # Chọn tập dữ liệu là "testing"
            sess
        )
    # Mở rộng chiều dữ liệu để phù hợp với đầu vào của mô hình, chuyển về dạng float32
    test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

    # Khởi tạo bộ thông dịch (interpreter) của TensorFlow Lite
    interpreter = tf.lite.Interpreter(
        tflite_model_path,
        experimental_op_resolver_type=tf.lite.experimental.OpResolverType.BUILTIN_REF
    )
    interpreter.allocate_tensors()  # Cấp phát bộ nhớ cho các tensor của mô hình

    # Lấy thông tin về tensor đầu vào và đầu ra của mô hình
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Nếu là mô hình lượng tử hóa (Quantized), cần chuyển đổi dữ liệu đầu vào về kiểu int (theo tỉ lệ lượng tử hóa)
    if model_type == "Quantized":
        input_scale, input_zero_point = input_details["quantization"]
        test_data = test_data / input_scale + input_zero_point
        test_data = test_data.astype(input_details["dtype"])  # Đảm bảo đúng kiểu dữ liệu int8

    correct_predictions = 0  # Biến đếm số lần dự đoán đúng
    for i in range(len(test_data)):
        # Đặt tensor đầu vào cho mô hình (mỗi lần 1 mẫu test)
        interpreter.set_tensor(input_details["index"], test_data[i])
        interpreter.invoke()  # Thực hiện suy luận (inference)
        output = interpreter.get_tensor(output_details["index"])[0]  # Lấy kết quả đầu ra
        top_prediction = output.argmax()  # Lấy nhãn có xác suất cao nhất
        correct_predictions += (top_prediction == test_labels[i])  # So sánh với nhãn thực tế

    # In ra kết quả độ chính xác của mô hình trên tập kiểm thử
    print('%s model accuracy is %f%% (Number of test samples=%d)' % (
        model_type, (correct_predictions * 100) / len(test_data), len(test_data)))


In [ ]:
# Tính toán độ chính xác của mô hình dạng float (số thực)
run_tflite_inference(FLOAT_MODEL_TFLITE)

# Tính toán độ chính xác của mô hình đã lượng tử hóa (quantized - dùng số nguyên 8 bit)
run_tflite_inference(MODEL_TFLITE, model_type='Quantized')


## Tạo mô hình TensorFlow Lite cho Vi điều khiển

Chuyển đổi mô hình TensorFlow Lite thành một tệp mã nguồn C có thể được nạp bởi TensorFlow Lite for Microcontrollers.


In [ ]:
# Cài đặt tiện ích xxd nếu chưa có (xxd dùng để chuyển file nhị phân sang dạng mã nguồn C)
!apt-get update && apt-get -qq install xxd

# Chuyển file mô hình .tflite sang dạng mã nguồn C (mảng unsigned char)
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

# Cập nhật tên biến trong file C cho phù hợp với tiêu chuẩn dự án (thường là g_model hoặc tên dễ nhớ)
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}


## Triển khai lên Vi điều khiển

Làm theo hướng dẫn trong [micro\_speech](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech) README.md cho [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview) để triển khai mô hình này lên một vi điều khiển cụ thể.

**Mô hình tham chiếu:** Nếu bạn chưa chỉnh sửa notebook này, bạn có thể làm theo hướng dẫn như bình thường để triển khai mô hình. Tham khảo thư mục [`micro_speech/train/models`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/train/models) để truy cập các mô hình được tạo trong notebook này.

**Mô hình mới:** Nếu bạn đã tạo một mô hình mới để nhận diện các từ khác: (i) Cập nhật `kCategoryCount` và `kCategoryLabels` trong [`micro_speech/micro_features/micro_model_settings.h`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.h) và (ii) Cập nhật các giá trị gán cho các biến được định nghĩa trong [`micro_speech/micro_features/model.cc`](https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/micro_speech/micro_features/model.cc) bằng các giá trị hiển thị sau khi chạy ô lệnh dưới đây.


In [ ]:
# In ra nội dung file mã nguồn C vừa tạo từ mô hình TFLite
!cat {MODEL_TFLITE_MICRO}